In [78]:
import anndata
import pandas as pd
import numpy as np
import scanpy as sp
import h5py
import zarr
import os
import tqdm
import time

from dredFISH.Analysis import basicu

In [79]:
%%time
f = '/bigstore/GeneralStorage/fangming/projects/dredfish/data/rna/scrna_10x_ctxhippo_a_DPNMF_matrix.h5ad'
adata = anndata.read(f, backed='r')
adata

CPU times: user 2.09 s, sys: 545 ms, total: 2.64 s
Wall time: 2.63 s


AnnData object with n_obs × n_vars = 1169213 × 24 backed at '/bigstore/GeneralStorage/fangming/projects/dredfish/data/rna/scrna_10x_ctxhippo_a_DPNMF_matrix.h5ad'
    obs: 'donor_sex_id', 'donor_sex_label', 'donor_sex_color', 'region_id', 'region_label', 'region_color', 'platform_label', 'cluster_order', 'cluster_label', 'cluster_color', 'subclass_order', 'subclass_label', 'subclass_color', 'neighborhood_id', 'neighborhood_label', 'neighborhood_color', 'class_order', 'class_label', 'class_color', 'exp_component_name', 'external_donor_name_label', 'full_genotype_label', 'facs_population_plan_label', 'injection_roi_label', 'injection_materials_label', 'injection_method_label', 'injection_type_label', 'full_genotype_id', 'full_genotype_color', 'external_donor_name_id', 'external_donor_name_color', 'facs_population_plan_id', 'facs_population_plan_color', 'injection_materials_id', 'injection_materials_color', 'injection_method_id', 'injection_method_color', 'injection_roi_id', 'injection_roi

In [80]:
cells_meta = adata.obs.index.values
cells_meta

array(['ACGCAGCAGACCGGAT-L8TX_180221_01_C11',
       'CCATGTCAGCGCTTAT-L8TX_180221_01_C11',
       'GCGCAGTAGTGTCTCA-L8TX_180712_01_H05', ...,
       'TTTGTCATCCTCAACC-L8TX_180115_01_E08',
       'TTTGTCATCGACAGCC-L8TX_180115_01_E08',
       'TTTGTCATCGTGGTCG-L8TX_180115_01_E08'], dtype=object)

In [81]:
f = '/bigstore/binfo/mouse/Brain/Sequencing/Allen_10X_CtxHippo/source/expression_matrix.hdf5'
with h5py.File(f, 'r') as fh:
    print(fh.keys())
    print(fh['data'].keys())
    
    # genes
    genes = fh['data']['gene'][()]
    genes = np.array([gene.decode() for gene in genes])
    print(genes.shape)
    
    # cells
    cells_inmat = fh['data']['samples'][()]
    cells_inmat = np.array([cell.decode() for cell in cells_inmat])
    print(cells_inmat.shape)
    
    # counts (actual data -- be careful)

<KeysViewHDF5 ['data']>
<KeysViewHDF5 ['counts', 'gene', 'samples', 'shape']>
(31053,)
(1169320,)


In [82]:
%%time
cellidx_in = np.sort(basicu.get_index_from_array(cells_inmat, cells_meta))
cellidx_in, cellidx_in.shape

CPU times: user 889 ms, sys: 124 ms, total: 1.01 s
Wall time: 1.01 s


(array([      0,       1,       2, ..., 1169317, 1169318, 1169319]),
 (1169213,))

In [83]:
cellname_in = cells_inmat[cellidx_in]
cellname_in, cellname_in.shape

(array(['AAACCTGAGAAACGCC-L8TX_180221_01_F09',
        'AAACCTGAGAAGGTTT-L8TX_180221_01_F09',
        'AAACCTGAGAGTAATC-L8TX_180221_01_F09', ...,
        'TTTGTCAGTGAGGCTA-L8TX_200611_02_A05',
        'TTTGTCATCAGGCGAA-L8TX_200611_02_A05',
        'TTTGTCATCGTCTGAA-L8TX_200611_02_A05'], dtype='<U35'),
 (1169213,))

In [84]:
dat_dir = '/bigstore/GeneralStorage/fangming/projects/dredfish/data/'
output = os.path.join(dat_dir, 'rna', 'scrna_10x_ctxhippo_a_exon_count_matrix_v2.zarr')
print(output)

/bigstore/GeneralStorage/fangming/projects/dredfish/data/rna/scrna_10x_ctxhippo_a_exon_count_matrix_v2.zarr


In [85]:
%%time
ncells = len(cellname_in)
ngenes = len(genes)
ncells, ngenes

CPU times: user 30 µs, sys: 4 µs, total: 34 µs
Wall time: 40.1 µs


(1169213, 31053)

In [86]:
%%time
# create a zarr file (dir)
chunksize = 100
z1 = zarr.open(output, mode='a')

# # create count matrix
# z1.create_dataset('counts', 
#                   shape=(ncells, ngenes), 
#                   chunks=(chunksize, None), 
#                   dtype='i4',
#                  )

CPU times: user 1.11 ms, sys: 4.13 ms, total: 5.24 ms
Wall time: 2.98 ms


In [10]:
z1['counts'].shape

(1169213, 31053)

In [14]:
%%time
readchunksize = 100000
curr_head = 0
with h5py.File(f, 'r') as fh:
    while tqdm.tqdm(curr_head < ncells):
        l, r = curr_head, curr_head+readchunksize
        r = min(ncells, r)
        curr_head += readchunksize
            
        subidx = cellidx_in[l:r]
        print(l,r, subidx.shape, subidx[0], subidx[-1])
        
        mat = fh['data']['counts'][:,subidx].T  # [gene, cell] -> [cell, gene]
        z1['counts'].oindex[l:r,:] = mat
        
        # break

0it [00:00, ?it/s]


0 100000 (100000,) 0 99999


0it [00:00, ?it/s]


100000 200000 (100000,) 100000 200014


0it [00:00, ?it/s]


...

1000000 1100000 (100000,) 1000107 1100106


0it [00:00, ?it/s]


1100000 1169213 (69213,) 1100107 1169319


0it [00:00, ?it/s]

CPU times: user 25min 55s, sys: 1min 16s, total: 27min 12s
Wall time: 25min 3s


In [33]:
z1['counts'], ncells

(<zarr.core.Array '/counts' (1169213, 31053) int32>, 1169213)

# Annotation

In [87]:
# create probe limit
z1.create_dataset(
    'num_probe_limit',
    shape=(ngenes,), 
    chunks=(chunksize,), 
    dtype='i4',
    overwrite=True,
)
z1['num_probe_limit'][:] = np.repeat([100], ngenes)

In [22]:
meta = adata.obs.copy()
meta = meta.loc[cellname_in]
meta

,donor_sex_id,donor_sex_label,donor_sex_color,region_id,region_label,region_color,platform_label,cluster_order,cluster_label,cluster_color,...,cell_type_accession_id,cell_type_designation_color,cell_type_alt_alias_color,cell_type_alias_color,cell_type_accession_color,cortical_layer_label,cortical_layer_order,cortical_layer_color,Lim1,Lim2
sample_name,,,,,,,,,,,,,,,,,,,,,
AAACCTGAGAAACGCC-L8TX_180221_01_F09,2,M,#ADC4C3,19,ACA,#FFDE17,10X,168,168_L2/3 IT CTX,#02F970,...,168,#02F970,#02F970,#02F970,#02F970,NaN,1,#FF7373,3.155724,-6.148659
AAACCTGAGAAGGTTT-L8TX_180221_01_F09,2,M,#ADC4C3,19,ACA,#FFDE17,10X,168,168_L2/3 IT CTX,#02F970,...,168,#02F970,#02F970,#02F970,#02F970,NaN,1,#FF7373,0.640433,-6.071972
AAACCTGAGAGTAATC-L8TX_180221_01_F09,2,M,#ADC4C3,19,ACA,#FFDE17,10X,189,189_L4/5 IT CTX,#09CCC6,...,189,#09CCC6,#09CCC6,#09CCC6,#09CCC6,NaN,1,#FF7373,5.952353,-1.614315
AAACCTGAGCCTTGAT-L8TX_180221_01_F09,2,M,#ADC4C3,19,ACA,#FFDE17,10X,290,290_L6 CT CTX,#3E98A5,...,290,#3E98A5,#3E98A5,#3E98A5,#3E98A5,NaN,1,#FF7373,15.818063,-3.473572
AAACCTGAGTGGTAAT-L8TX_180221_01_F09,2,M,#ADC4C3,19,ACA,#FFDE17,10X,265,265_L5/6 NP CTX,#45BE72,...,265,#45BE72,#45BE72,#45BE72,#45BE72,NaN,1,#FF7373,14.517274,2.286561
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTCAGTCGCGGTT-L8TX_200611_02_A05,1,F,#565353,21,TEa-PERI-ECT,#76CBE2,10X,18,18_Lamp5,#D97C80,...,18,#D97C80,#D97C80,#D97C80,#D97C80,NaN,1,#FF7373,5.398016,12.946978
TTTGTCAGTCTAGGTT-L8TX_200611_02_A05,1,F,#565353,21,TEa-PERI-ECT,#76CBE2,10X,265,265_L5/6 NP CTX,#45BE72,...,265,#45BE72,#45BE72,#45BE72,#45BE72,NaN,1,#FF7373,14.953689,3.713908
TTTGTCAGTGAGGCTA-L8TX_200611_02_A05,1,F,#565353,21,TEa-PERI-ECT,#76CBE2,10X,162,162_L2/3 IT CTX,#0E993B,...,162,#0E993B,#0E993B,#0E993B,#0E993B,NaN,1,#FF7373,0.225908,-7.700701


In [23]:
# create labels
z1.create_dataset(
    'l1_code',
    shape=(ncells,), 
    chunks=(chunksize,), 
    dtype='i4',
)
a = meta['class_label']
z1['l1_code'] = a.cat.codes.values
z1['l1_cat'] = a.cat.categories.values.astype(str)

z1.create_dataset(
    'l2_code',
    shape=(ncells,), 
    chunks=(chunksize,), 
    dtype='i4',
)
a = meta['neighborhood_label']
z1['l2_code'] = a.cat.codes.values
z1['l2_cat'] = a.cat.categories.values.astype(str)

z1.create_dataset(
    'l3_code',
    shape=(ncells,), 
    chunks=(chunksize,), 
    dtype='i4',
)
a = meta['subclass_label']
z1['l3_code'] = a.cat.codes.values
z1['l3_cat'] = a.cat.categories.values.astype(str)

z1.create_dataset(
    'l5_code',
    shape=(ncells,), 
    chunks=(chunksize,), 
    dtype='i4',
)
a = meta['cluster_label']
z1['l5_code'] = a.cat.codes.values
z1['l5_cat'] = a.cat.categories.values.astype(str)

In [29]:
%%time
y = z1['counts'].oindex[975,:]
y, y.shape

CPU times: user 11.7 ms, sys: 12 ms, total: 23.7 ms
Wall time: 12.7 ms


(array([9, 1, 0, ..., 0, 0, 0], dtype=int32), (31053,))

In [25]:
%%time
testidx = np.random.choice(ncells, 100)

CPU times: user 793 µs, sys: 4.07 ms, total: 4.86 ms
Wall time: 7.52 ms


In [30]:
%%time
z1['counts'].oindex[testidx,:].shape

CPU times: user 2.32 s, sys: 190 ms, total: 2.51 s
Wall time: 1.44 s


(100, 31053)

In [31]:
%%time
z1['counts'].oindex[testidx[0],:].shape

CPU times: user 11.3 ms, sys: 4.67 ms, total: 16 ms
Wall time: 5.88 ms


(31053,)

In [32]:
%%time
z1['l3_code'].oindex[testidx[0]].shape

CPU times: user 6.86 ms, sys: 0 ns, total: 6.86 ms
Wall time: 3.9 ms


()

# split train and test

In [34]:
def split_train_test(zarr_file, keys_copy, keys_split, frac=0.9, random_seed=None):
    """randomly select frac vs 1-frac samples into training and test (validation) set.
    Save them as separate zarr files
    """
    assert frac <= 1 and frac >= 0
    # the original zarr file
    z = zarr.open(zarr_file, 'r')
    size = len(z['counts'])
    
    path_train = zarr_file.replace('.zarr', '_train.zarr')
    path_test = zarr_file.replace('.zarr', '_test.zarr')
    logging.info(f"{zarr_file} -> \n{path_train} and \n{path_test}\n")
    if random_seed: np.random.seed(random_seed)
    cond_train = np.random.rand(size) < frac
    ntrain = cond_train.sum()
    ntest = (~cond_train).sum()
    logging.info(f"{size}, {ntrain} ({ntrain/size:.3f}), {ntest} ({ntest/size:.3f})")
    
    z_train = zarr.open(path_train, mode='w')
    z_test = zarr.open(path_test, mode='w')
    for key in keys_copy:
        z_train[key] = z[key]
        z_test[key] = z[key]

    for key in keys_split:
        # train
        if z[key].ndim == 1:
            chunksize = (10,)
            dat = z[key].oindex[cond_train]
        elif z[key].ndim == 2:
            chunksize = (10,None)
            dat = z[key].oindex[cond_train,:]
        else:
            raise ValueError('unimplemented')
            
        z_train.create_dataset(key, shape=dat.shape, chunks=chunksize)
        z_train[key][:] = dat
        
        # test
        if z[key].ndim == 1:
            chunksize = (10,)
            dat = z[key].oindex[~cond_train]
        elif z[key].ndim == 2:
            chunksize = (10,None)
            dat = z[key].oindex[~cond_train,:]
        else:
            raise ValueError('unimplemented')
            
        z_test.create_dataset(key, shape=dat.shape, chunks=chunksize)
        z_test[key][:] = dat
        
    return 

In [36]:
z = zarr.open(output, mode='r')
z, list(z.keys())

(<zarr.hierarchy.Group '/' read-only>,
 ['counts',
  'l1_cat',
  'l1_code',
  'l2_cat',
  'l2_code',
  'l3_cat',
  'l3_code',
  'l5_cat',
  'l5_code',
  'num_probe_limit'])

In [37]:
#
keys_copy = ['num_probe_limit', 'l1_cat', 'l2_cat', 'l3_cat', 'l5_cat']
keys_split = [key for key in z.keys() if key not in keys_copy]
keys_copy, keys_split


(['num_probe_limit', 'l1_cat', 'l2_cat', 'l3_cat', 'l5_cat'],
 ['counts', 'l1_code', 'l2_code', 'l3_code', 'l5_code'])

In [38]:
%%time
# split_train_test(output, keys_copy, keys_split, frac=0.9)

CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 16 µs


# test if we can load train and val as well as the original

In [6]:
# print(output)

In [69]:
f_org = '/bigstore/GeneralStorage/fangming/projects/dredfish/data/rna/scrna_10x_ctxhippo_a_exon_count_matrix_v2.zarr'
f_trn = '/bigstore/GeneralStorage/fangming/projects/dredfish/data/rna/scrna_10x_ctxhippo_a_exon_count_matrix_v2_train.zarr'
f_tst = '/bigstore/GeneralStorage/fangming/projects/dredfish/data/rna/scrna_10x_ctxhippo_a_exon_count_matrix_v2_test.zarr'

f_org2 = '/bigstore/GeneralStorage/fangming/projects/dredfish/data/rna/scrna_ss_ctxhippo_a_exon_count_matrix_v3.zarr'
f_trn2 = '/bigstore/GeneralStorage/fangming/projects/dredfish/data/rna/scrna_ss_ctxhippo_a_exon_count_matrix_v3_train.zarr'


In [72]:
z = zarr.open(f_org, mode='r')
for key in z.keys():
    print(z[key])
print("===")
    
zt = zarr.open(f_trn, mode='r')
for key in zt.keys():
    print(zt[key])
print("===")

ztst = zarr.open(f_tst, mode='r')
for key in ztst.keys():
    print(ztst[key])
print("===")


z2 = zarr.open(f_org2, mode='r')
for key in z2.keys():
    print(z2[key])
print("===")
    
zt2 = zarr.open(f_trn2, mode='r')
for key in zt2.keys():
    print(zt2[key])
print("===")

<zarr.core.Array '/counts' (1169213, 31053) int32 read-only>
<zarr.core.Array '/l1_cat' (3,) <U13 read-only>
<zarr.core.Array '/l1_code' (1169213,) int8 read-only>
<zarr.core.Array '/l2_cat' (8,) <U14 read-only>
<zarr.core.Array '/l2_code' (1169213,) int8 read-only>
<zarr.core.Array '/l3_cat' (42,) <U15 read-only>
<zarr.core.Array '/l3_code' (1169213,) int8 read-only>
<zarr.core.Array '/l5_cat' (387,) <U23 read-only>
<zarr.core.Array '/l5_code' (1169213,) int16 read-only>
<zarr.core.Array '/num_probe_limit' (1169213,) int32 read-only>
===
<zarr.core.Array '/counts' (1052202, 31053) float64 read-only>
<zarr.core.Array '/l1_cat' (3,) <U13 read-only>
<zarr.core.Array '/l1_code' (1052202,) float64 read-only>
<zarr.core.Array '/l2_cat' (8,) <U14 read-only>
<zarr.core.Array '/l2_code' (1052202,) float64 read-only>
<zarr.core.Array '/l3_cat' (42,) <U15 read-only>
<zarr.core.Array '/l3_code' (1052202,) float64 read-only>
<zarr.core.Array '/l5_cat' (387,) <U23 read-only>
<zarr.core.Array '/l5_c

In [64]:
size = len(z['counts'])
cond_train = np.random.rand(size) < 0.9
# cond_train.sum(),(~cond_train).sum()
trn_idx = np.arange(size)[cond_train]
tst_idx = np.arange(size)[~cond_train]
len(trn_idx), len(tst_idx)

(1052217, 116996)

In [35]:
%%time
z['counts'].oindex[12356,:]

CPU times: user 31.4 ms, sys: 0 ns, total: 31.4 ms
Wall time: 8.06 ms


array([11,  0,  0, ...,  0,  0,  0], dtype=int32)

In [36]:
%%time
z2['counts'].oindex[12356,:]

CPU times: user 6.36 ms, sys: 6.06 ms, total: 12.4 ms
Wall time: 6.23 ms


array([0, 0, 2, ..., 0, 0, 0], dtype=int32)

In [53]:
%%time
nsample = 100
ti = time.time()
for i in range(nsample):
    idx = np.random.choice(100000,size=1)
    z['counts'].oindex[idx,:]
    z['l3_code'].oindex[idx,]
print(time.time()-ti)

ti = time.time()
for i in range(nsample):
    idx = np.random.choice(10000,size=1)
    z2['counts'].oindex[idx,:]
    z2['l3_code'].oindex[idx,]
print(time.time()-ti)

ti = time.time()
for i in range(nsample):
    idx = np.random.choice(100000,size=1)
    zt['counts'].oindex[idx,:]
    zt['l3_code'].oindex[idx,]
print(time.time()-ti)

ti = time.time()
for i in range(nsample):
    idx = np.random.choice(10000,size=1)
    zt2['counts'].oindex[idx,:]
    zt2['l3_code'].oindex[idx,]
print(time.time()-ti)

5.338320970535278
3.1829171180725098
4.032543659210205
4.913153886795044
CPU times: user 11.2 s, sys: 1.19 s, total: 12.4 s
Wall time: 17.5 s


In [38]:
z_trn = zarr.open(f_trn, mode='r')
z_tst = zarr.open(f_tst, mode='r')
z_trn, list(z_trn.keys()), z_tst, list(z_tst.keys())

(<zarr.hierarchy.Group '/' read-only>,
 ['counts',
  'l1_cat',
  'l1_code',
  'l2_cat',
  'l2_code',
  'l3_cat',
  'l3_code',
  'l5_cat',
  'l5_code',
  'num_probe_limit'],
 <zarr.hierarchy.Group '/' read-only>,
 ['counts',
  'l1_cat',
  'l1_code',
  'l2_cat',
  'l2_code',
  'l3_cat',
  'l3_code',
  'l5_cat',
  'l5_code',
  'num_probe_limit'])

In [50]:
zt['l3_code'].oindex[:100]

array([11., 11., 17., 23., 22., 13., 17., 28., 11., 11., 23., 18., 24.,
       38., 23., 23., 23., 23., 11., 24., 26., 23., 23., 17., 17.,  2.,
       28., 23., 41., 23., 34., 28., 13., 28., 24., 23., 23., 28., 11.,
       23., 38., 18., 23., 13., 28., 23., 11., 11., 24., 17., 24., 17.,
       23., 23., 18., 23., 34., 39., 18., 23., 17., 24., 41., 24., 18.,
       41., 20., 17., 11., 18., 34., 11., 26., 22., 23., 11., 11., 11.,
       22., 11., 13., 17., 20., 41., 23., 23., 18., 28., 23., 26., 24.,
       11., 38., 17., 23., 18., 24., 28., 11., 11.])

In [39]:
z_trn['num_probe_limit'][:]

array([100, 100, 100, ..., 100, 100, 100], dtype=int32)

In [40]:
%%time
y = z_trn['counts'].oindex[975,:]
y, y.shape

CPU times: user 2.77 ms, sys: 2.07 ms, total: 4.83 ms
Wall time: 11.8 ms


(array([6.0200e+02, 3.3650e+03, 6.8570e+03, 5.0500e+02, 3.3870e+03,
        4.1500e+02, 6.2000e+02, 2.5900e+02, 8.7100e+02, 1.6480e+03,
        3.0190e+03, 2.6210e+03, 1.9860e+03, 7.6290e+03, 3.4400e+02,
        7.0000e+00, 4.0830e+03, 1.1760e+03, 1.0560e+03, 8.5000e+01,
        1.0506e+04, 9.5800e+02, 7.5900e+02, 9.6510e+03]),
 (24,))

In [41]:
%%time
y = z_tst['counts'].oindex[975,:]
y, y.shape

CPU times: user 2.71 ms, sys: 1.98 ms, total: 4.69 ms
Wall time: 4.02 ms


(array([ 1474.,  5740., 13134.,  1021.,  5632.,  1386.,   886.,   460.,
         1047.,  4762.,  6043.,  5104.,  4258., 10737.,   569.,    39.,
         8308.,  2077.,  2559.,   248., 15935.,  1580.,  1976., 13812.]),
 (24,))

In [47]:
z['num_probe_limit'][:10] #, z_trn['num_probe_limit'][:10], z_tst['num_probe_limit'][:10]

array([100, 100, 100, 100, 100, 100, 100, 100, 100, 100], dtype=int32)